In [1]:
from azure.quantum import Workspace

In [2]:
workspace = Workspace(
    resource_id = "/subscriptions/a0f970ef-72e9-4075-8009-32c19007a132/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/QCE24-QRE-Challenge",
    location = "southuk"
    )

In [3]:
from azure.quantum.qiskit import AzureQuantumProvider
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RGQFTMultiplier

In [4]:
provider = AzureQuantumProvider(workspace)
backend = provider.get_backend('microsoft.estimator')

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/wmk/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/wmk/snap/code/common/.cache/gio-modules/libgiolibproxy.so
[87658:87658:0709/092000.759889:ERROR:viz_main_impl.cc(196)] Exiting GPU process due to errors during initialization
[87806:87806:0709/092001.009233:ERROR:viz_main_impl.cc(196)] Exiting GPU process due to errors during initialization
[87776:7:0709/092001.105845:ERROR:command_buffer_proxy_impl.cc(127)] ContextResult::kTransientFailure: Failed to send GpuControl.CreateCommandBuffer.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
_DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credenti

ClientAuthenticationError: _DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	_TokenFileCredential: Token file location not set.
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	VisualStudioCodeCredential: Failed to get Azure user details from Visual Studio Code. Currently, the VisualStudioCodeCredential only works with the Azure Account extension version 0.9.11 and earlier. A long-term fix is in progress, see https://github.com/Azure/azure-sdk-for-python/issues/25713
	AzureCliCredential: Azure CLI not found on path
	AzurePowerShellCredential: PowerShell is not installed
	InteractiveBrowserCredential: Authentication failed: access_denied

In [ ]:
def create_algorithm(bitwidth):
    print(f"[INFO] Create a QFT-based multiplier with bitwidth {bitwidth}")
    
    # Print a warning for large bitwidths that will require some time to generate and
    # transpile the circuit.
    if bitwidth > 18:
        print(f"[WARN] It will take more than one minute generate a quantum circuit with a bitwidth larger than 18")

    circ = RGQFTMultiplier(num_state_qubits=bitwidth, num_result_qubits=2 * bitwidth)

    # One could further reduce the resource estimates by increasing the optimization_level,
    # however, this will also increase the runtime to construct the algorithm.  Note, that
    # it does not affect the runtime for resource estimation.
    print(f"[INFO] Decompose circuit into intrinsic quantum operations")

    circ = transpile(circ, basis_gates=SUPPORTED_INSTRUCTIONS, optimization_level=0)

    # print some statistics
    print(f"[INFO]   qubit count: {circ.num_qubits}")
    print("[INFO]   gate counts")
    for gate, count in circ.count_ops().items():
        print(f"[INFO]   - {gate}: {count}")

    return circ

In [ ]:
bitwidth = 4

circ = create_algorithm(bitwidth)

In [ ]:
job = backend.run(circ)
result = job.result()
result